### Import necessary names
Always run the cell below after kernel restart.

In [ ]:
from pybeam.datamodels import nodes_from_csv, elements_from_csv
from pybeam.assembly import assemble_system_matrices
from pybeam.simulation import newmark
from pybeam.modal_parameters import get_modal_parameters
from pybeam._utilities import pprint_array

import numpy as np

### Initialize system matrices

In [ ]:
nodes = nodes_from_csv("data/nodes.csv")
elements = elements_from_csv("data/elements.csv", nodes)

stiffness, mass = assemble_system_matrices(elements)
damping = 0.01 * mass + 0.001 * stiffness
# damping = None

In [ ]:
eigenfrequencies, modeshapes = get_modal_parameters(stiffness, mass)
pprint_array(eigenfrequencies)

### Set simulation parameters

In [ ]:
# Number of DOF
n_dof = stiffness.shape[0]

# Time step size
dt = 0.0001

# End time
t_end = 10
# Time steps
time = np.arange(start=0, stop=t_end, step=dt)

# Loading
loads = np.zeros((n_dof, len(time)))
loads[-2, :] = 100 *  np.cos(
    (eigenfrequencies[0] + eigenfrequencies[1]) / 2
    * time
)

# Initial conditions
x_0 = np.zeros((n_dof,))
# x_0[-2] = 0.1
v_0 = np.zeros((n_dof,))

### Run simulation

In [ ]:
x, v, a = newmark(
    stiffness=stiffness,
    mass=mass,
    damping=damping,
    initial_disp=x_0,
    initial_vel=v_0,
    loads=loads,
    time=time,
    beta=1/6,
)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib
plt.ion()
plt.style.use("dark_background")


ax = plt.gca()
ax.plot(time, x[-2, :])

plt.show()